# Bayesian Symbolic Regression (BSR)

## Introduction

Symbolic regression (SR) refers to a class of algorithms that search for interpretable symbolic expressions which capture relationships within data. For a general introduction into SR, see the docs for [Bayesian Machine Scientist (BMS)](https://github.com/AutoResearch/autora/blob/main/docs/theorist/bms/introduction.md). Bayesian Symbolic Regression, proposed by [Jin et. al (2019)](https://arxiv.org/abs/1910.08892), is a method that fits SR with interpretable and concise expressions under a Bayesian framework. Following are its main contributions in a step-by-step order:

1. It models an interpretable symbolic expression as an expression tree -- with root and intermediate tree nodes representing operators (e.g. `*` for a binary node and `sin` for an unary node) and leaf nodes representing a feature in the data. BSR then defines the search space as the union of three parts
    - Tree Structure (T): this part is about the structure of the expression tree (e.g. how to recursively construct the tree and when to stop by using leaf nodes). It also specifies the assignment of operators to non-leaf nodes.
    - Leaf Nodes (M): this part assigns features to leaf nodes that are already defined from part **T**.
    - Operator parameters ($\Theta$): certain operators (e.g. a linear operator `ln` with intercept and slope params) have additional parameters to define. We collect these parameters together in a vector $\Theta$.

2. It specifies the priors for each of the three spaces above. In `Autora`'s implementation of BSR, you have the freedom of specifying your own prior for part `T`, or choosing among the pre-specified priors.

3. It defines the `actions` that enable transition of one expression tree (`original`) into a new expression tree (`proposed`), and supports the calculation of transition probabilities based on the likelihoods of `original` and `proposed` models.

4. It designs and implements a Reversible-Jump Markov-Chain Monte-Carlo (RJ-MCMC) algorithm that iteratively accepts new samples (each sample is a valid expression tree) based on the transition probabilities calculated above. In each iteration, `K` expression trees are obtained either from the `original` samples or the newly `proposed` samples.

5. In each iteration, the final prediction model is a linear mixture of the `K` trees, where we regress the ground truth response on the `K` expression results generated by the trees to obtain the linear regression parameters $\beta$.

`AutoRA`'s implementation of BSR is adapted from original authors' [codebase](https://github.com/ying531/MCMC-SymReg), with fully refactored codes on data structures and MCMC computations. We also provide new sets of priors that suit applications in human information processing.

<br>

## Meta-Parameters

Meta-Parameters are used to control the search space and the model configuration. In BSR, they are mainly defined in the theorist (see `bsr.py`) constructor. Below are a basic overview of these parameters (Note that there are additional algorithm-irrelevant configurations that can be customized in the constructor. Please refer to code documentation for their details).

- `tree_num`: also known as `K` in BSR, the number of expression trees to use in the linear mixture (final prediction model).
- `iter_num`: the number of RJ-MCMC steps to run in BSR. Can be also understood as the number of `K`-samples to take in the whole fitting process.
- `val`: the number of validation steps to run following each iteration step.
- `beta`: the hyperparameter that controls growing of a new expression tree. It needs to be < 0. In general, smaller value of `beta` corresponds to generating deeper expression trees.

<br>

## Search Space

Here we present the list of in-built operators in BSR.

- **\+**: The output of the computation $x_j$ is the sum over its two inputs $x_i, x_{ii}$: $x_j = x_i + x_{ii}$.
- **\-**: The output of the computation $x_j$ is the respective difference between its inputs $x_i, x_{ii}$: $x_j = x_i - x_{ii}$.
- **\***: The output of the computation $x_j$ is the product over its two inputs $x_i, x_{ii}$: $x_j = x_i * x_{ii}$.
- **exp**: The output of the computation $x_j$ is the natural exponential function applied to its input $x_i$: $x_j = \exp(x_i)$.
- **pow2**: The output of the computation $x_j$ is the square function applied to its input $x_i$: $x_j$ = $x_i^2$.
- **pow3**: The output of the computation $x_j$ is the cube function applied to its input $x_i$: $x_j$ = $x_i^3$.
- **sin**: The output of the computation $x_j$ is the sine function applied to its input $x_i$: $x_j = \sin(x_i)$.
- **cos**: The output of the computation $x_j$ is the cosine function applied to its input $x_i$: $x_j = \cos(x_i)$.
- **ln**: The output of the computation $x_j$ is the linear transformation applied to its input $x_i$: $x_j = a * x_i + b$. $a$ and $b$ are the slope and intercept parameters associated with the operator.

In BSR, it's easy to add an operator to the existing operators by following two steps. Firstly, define an operator as a function similar to examples in `operations.py`, and then add the operator's name and prior information to the dictionaries in `__get_prior()` within `prior.py`.

<br>

## References
Jin, Ying, et al. "Bayesian symbolic regression." arXiv preprint arXiv:1910.08892 (2019).
